In [1]:
import ccxt
import os
import datetime
import math

from dotenv import dotenv_values, load_dotenv
from pprint import pprint
from arch import arch_model

# import ccxt.async_support as ccxt # link against the asynchronous version of ccxt

from functions.BS_pricer import BS_pricer
from functions.Parameters import Option_param
from functions.Processes import Diffusion_process

import numpy as np
import pandas as pd
import scipy as scp

# Set the float_format to limit the number of decimal places to 3
pd.options.display.float_format = '{:,.3f}'.format

import scipy.stats as ss
from scipy.integrate import quad
from functools import partial

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load environment variables from .env file
load_dotenv()

"""
# Set the API keys for the exchanges
delta_api_key = os.getenv('DELTA_API_KEY')
delta_api_secret = os.getenv('DELTA_API_SECRET')
deribit_api_key = os.getenv('DERIBIT_API_KEY')
deribit_api_secret = os.getenv('DERIBIT_API_SECRET')
"""

exchanges_list = [
    "delta",
    "deribit",
    "binance",
    "huobi",
    "okex",
]
exchanges_dict = {}
for exch in exchanges_list:
    exchanges_dict[exch] = getattr(ccxt, exch)({
        'apiKey': os.getenv(f"{exch.upper()}_API_KEY") or None,
        'secret': os.getenv(f"{exch.upper()}_API_SECRET") or None,
    })
"""    
# Create the exchange objects
delta = ccxt.delta({
    'apiKey': delta_api_key,
    'secret': delta_api_secret,
})
deribit = ccxt.deribit({
    'apiKey': deribit_api_key,
    'secret': deribit_api_secret,
})
binance = ccxt.binance({
    'apiKey': binance_api_key,
    'secret': binance_api_secret,
})
huobi = ccxt.huobi({
    'apiKey': huobi_api_key,
    'secret': huobi_api_secret,
})
okex = ccxt.okex({
    'apiKey': okex_api_key,
    'secret': okex_api_secret,
})
# Create the dictionary
exchanges_dict = {'delta': delta, 'deribit': deribit}
pprint(exchanges_dict)
"""

"    \n# Create the exchange objects\ndelta = ccxt.delta({\n    'apiKey': delta_api_key,\n    'secret': delta_api_secret,\n})\nderibit = ccxt.deribit({\n    'apiKey': deribit_api_key,\n    'secret': deribit_api_secret,\n})\nbinance = ccxt.binance({\n    'apiKey': binance_api_key,\n    'secret': binance_api_secret,\n})\nhuobi = ccxt.huobi({\n    'apiKey': huobi_api_key,\n    'secret': huobi_api_secret,\n})\nokex = ccxt.okex({\n    'apiKey': okex_api_key,\n    'secret': okex_api_secret,\n})\n# Create the dictionary\nexchanges_dict = {'delta': delta, 'deribit': deribit}\npprint(exchanges_dict)\n"

In [3]:
def binomial_price(S0, K, T, r, sigma, N=15000, payoff="call"):
    # N number of periods or number of time steps  

    dT = float(T) / N                             # Delta t
    u = np.exp(sigma * np.sqrt(dT))                 # up factor
    d = 1.0 / u                                   # down factor 

    V = np.zeros(N+1)                             # initialize the price vector
    S_T = np.array( [(S0 * u**j * d**(N - j)) for j in range(N + 1)] )  # price S_T at time T

    a = np.exp(r * dT)    # risk free compounded return
    p = (a - d)/ (u - d)  # risk neutral up probability
    q = 1.0 - p           # risk neutral down probability   

    if payoff =="call":
        V[:] = np.maximum(S_T-K, 0.0)
    else:
        V[:] = np.maximum(K-S_T, 0.0)

    for i in range(N-1, -1, -1):
        V[:-1] = np.exp(-r*dT) * (p * V[1:] + q * V[:-1])    # the price vector is overwritten at each step
            
    return V[0]


In [4]:
# delta_instruments = exchanges_dict['delta'].fetchMarkets()
# deribit_instruments = exchanges_dict['deribit'].fetchMarkets()
instruments_dict = {k: exchanges_dict[k].fetchMarkets() for k in exchanges_dict}
instruments_dict['delta'] = {d['symbol']: d for d in instruments_dict['delta']}
instruments_dict['deribit'] = {d['symbol']: d for d in instruments_dict['deribit']}

In [24]:
# We define the contract we are looking for
contract_specification = {
    'expiration': '230128',
    'payoff': 'C',
    'underlying': 'BTC',
    'strike': 23500,
    'quote': 'USDT',
    'side': 'bid',
}

In [39]:
def get_ticker(strike, expiration, underlying, quote, payoff, side):
    # Construct the ticker symbol for the contract
    delta_ticker_specification = f"{underlying}/{quote}:{quote}-{expiration}:{strike}:{payoff}"
    if quote == "USDT": quote = "USD"
    deribit_ticker_specification = f"{underlying}/{quote}:{underlying}-{expiration}-{strike}-{payoff}"
    # Fetch the ticker data from the Delta exchange
    delta_ticker = exchanges_dict['delta'].fetch_ticker(delta_ticker_specification)
    pprint(f"Delta ticker: {delta_ticker}")

    # Fetch the ticker data from the Deribit exchange
    deribit_ticker = exchanges_dict['deribit'].fetch_ticker(deribit_ticker_specification)
    pprint(f"Deribit ticker: {deribit_ticker}")

    return {
        "delta": delta_ticker,
        "deribit": deribit_ticker,
    }
    
tickers_dict = get_ticker(**contract_specification)

("Delta ticker: {'symbol': 'BTC/USDT:USDT-230128:23500:C', 'timestamp': "
 "1674743928859, 'datetime': '2023-01-26T14:38:48.859Z', 'high': 157.5, 'low': "
 "157.5, 'bid': None, 'bidVolume': None, 'ask': None, 'askVolume': None, "
 "'vwap': 23061.845699565216, 'open': 157.5, 'close': 157.5, 'last': 157.5, "
 "'previousClose': None, 'change': 0.0, 'percentage': 0.0, 'average': 157.5, "
 "'baseVolume': 0.345, 'quoteVolume': 7956.33676635, 'info': {'close': "
 "'157.5', 'contract_type': 'call_options', 'greeks': {'delta': '0.34891488', "
 "'gamma': '0.00044192', 'rho': '0.40836504', 'spot': '23157.307369156802', "
 "'theta': '-81.95495567', 'vega': '6.16484272'}, 'high': '157.5', 'low': "
 "'157.5', 'mark_price': '194.44008846', 'mark_vol': "
 "'0.5028292560018599033355712894', 'oi': '0.3450', 'oi_contracts': '345', "
 "'oi_value': '0.3450', 'oi_value_symbol': 'BTC', 'oi_value_usd': '7993.2595', "
 "'open': '157.5', 'price_band': {'lower_limit': '0.100000000000000000', "
 "'upper_limit': '

In [40]:
def parse_option_spectification_delta(option_specification):
    expiration = datetime.datetime.strptime(option_specification['expiration'], "%y%m%d")
    delta_expiration = datetime.datetime.strptime(option_specification['expiration']+"125959", "%y%m%d%H%M%S")
    deribit_expiration = datetime.datetime.strptime(option_specification['expiration']+"075959", "%y%m%d%H%M%S")
    return {
        "underlying": option_specification['underlying'],
        "quote": option_specification['quote'],
        "expiration": expiration,
        "strike": float(option_specification['strike']),
        "payoff": "call" if option_specification['payoff'] in ("C", "call", "Call", "CALL") else "put",
        "side": "ask" if option_specification['side'] in ("a", "ask", "Ask", "ASK") else "bid",
        "delta_expiration": delta_expiration,
        "deribit_expiration": deribit_expiration,
    }

option_specification = parse_option_spectification_delta(contract_specification)
pprint(option_specification)

{'delta_expiration': datetime.datetime(2023, 1, 28, 12, 59, 59),
 'deribit_expiration': datetime.datetime(2023, 1, 28, 7, 59, 59),
 'expiration': datetime.datetime(2023, 1, 28, 0, 0),
 'payoff': 'call',
 'quote': 'USDT',
 'side': 'bid',
 'strike': 23500.0,
 'underlying': 'BTC'}


In [41]:
now = datetime.datetime.now()
forward_date = option_specification["expiration"]
delta_forward_date = option_specification["delta_expiration"]
deribit_forward_date = option_specification["deribit_expiration"]

print(f"Delta: {(now, delta_forward_date)}")
print(f"Deribit: {(now, deribit_forward_date)}")

timedelta_until_expiration = forward_date - now
delta_timedelta_until_expiration = delta_forward_date - now
deribit_timedelta_until_expiration = deribit_forward_date - now

days_until_expiration = timedelta_until_expiration.total_seconds() / 86400
delta_days_until_expiration = delta_timedelta_until_expiration.total_seconds() / 86400
deribit_days_until_expiration = deribit_timedelta_until_expiration.total_seconds() / 86400

print(days_until_expiration)
print(delta_days_until_expiration)
print(deribit_days_until_expiration)

Delta: (datetime.datetime(2023, 1, 26, 15, 38, 52, 149048), datetime.datetime(2023, 1, 28, 12, 59, 59))
Deribit: (datetime.datetime(2023, 1, 26, 15, 38, 52, 149048), datetime.datetime(2023, 1, 28, 7, 59, 59))
1.3480075341666666
1.8896626267592593
1.681329293425926


In [42]:
pprint(tickers_dict['delta']['symbol'])

# pprint(instruments_dict['delta'][tickers_dict['delta']['symbol']])
delta_maker_fees_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['maker'])*100
delta_taker_fees_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['taker'])*100
delta_premium_commission_rate = float(instruments_dict['delta'][tickers_dict['delta']['symbol']]['info']['product_specs']['premium_commission_rate'])

pprint([
    delta_maker_fees_rate,
    delta_taker_fees_rate,
    delta_premium_commission_rate
])

'BTC/USDT:USDT-230128:23500:C'
[0.03, 0.03, 0.1]


In [43]:
pprint(tickers_dict['deribit']['symbol'])
# pprint(instruments_dict['deribit'][tickers_dict['deribit']['symbol']])

deribit_maker_fees_rate = float(instruments_dict['deribit'][tickers_dict['deribit']['symbol']]['maker'])*100
deribit_taker_fees_rate = float(instruments_dict['deribit'][tickers_dict['deribit']['symbol']]['taker'])*100
deribit_premium_commission_rate = 0.125

pprint([
    deribit_maker_fees_rate,
    deribit_taker_fees_rate,
    deribit_premium_commission_rate
])

'BTC/USD:BTC-230128-23500-C'
[0.03, 0.03, 0.125]


In [44]:
"""
try:
    a = exchanges_dict['delta'].fetchFees()
    pprint(f"fetchFees: {a}")
    #b = exchanges_dict['delta'].fetchTradingFee(tickers_dict['delta']['info']['product_id'])
    #pprint(f"fetchFees: {b}")
    #c = exchanges_dict['delta'].fetchTradingFees()
    #pprint(f"fetchFees: {c}")
    # d = exchanges_dict['delta'].fetchTransactionFee()
    # pprint(f"fetchFees: {d}")
    #e = exchanges_dict['delta'].fetchTransactionFees()
    #pprint(f"fetchFees: {e}")
    #a = exchanges_dict['deribit'].fetchFees()
    #pprint(f"fetchFees: {a}")
    b = exchanges_dict['deribit'].fetchTradingFee(tickers_dict['deribit']['info']['instrument_name'])
    pprint(f"fetchFees: {b}")
    c = exchanges_dict['deribit'].fetchTradingFees()
    pprint(f"fetchFees: {c}")
    d = exchanges_dict['deribit'].fetchTransactionFee()
    pprint(f"fetchFees: {d}")
    e = exchanges_dict['deribit'].fetchTransactionFees()
    pprint(f"fetchFees: {e}")
except Exception as e:
    pprint(e)
"""

'\ntry:\n    a = exchanges_dict[\'delta\'].fetchFees()\n    pprint(f"fetchFees: {a}")\n    #b = exchanges_dict[\'delta\'].fetchTradingFee(tickers_dict[\'delta\'][\'info\'][\'product_id\'])\n    #pprint(f"fetchFees: {b}")\n    #c = exchanges_dict[\'delta\'].fetchTradingFees()\n    #pprint(f"fetchFees: {c}")\n    # d = exchanges_dict[\'delta\'].fetchTransactionFee()\n    # pprint(f"fetchFees: {d}")\n    #e = exchanges_dict[\'delta\'].fetchTransactionFees()\n    #pprint(f"fetchFees: {e}")\n    #a = exchanges_dict[\'deribit\'].fetchFees()\n    #pprint(f"fetchFees: {a}")\n    b = exchanges_dict[\'deribit\'].fetchTradingFee(tickers_dict[\'deribit\'][\'info\'][\'instrument_name\'])\n    pprint(f"fetchFees: {b}")\n    c = exchanges_dict[\'deribit\'].fetchTradingFees()\n    pprint(f"fetchFees: {c}")\n    d = exchanges_dict[\'deribit\'].fetchTransactionFee()\n    pprint(f"fetchFees: {d}")\n    e = exchanges_dict[\'deribit\'].fetchTransactionFees()\n    pprint(f"fetchFees: {e}")\nexcept Exception

In [45]:
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
# since = exchange.milliseconds () - 86400000  # -1 day from now
start_date = '2022-01-09T00:00:00Z' # fetch from a certain starting datetime
t_frame = '1d' # 1-day timeframe, usually from 1-minute to 1-week depending on the exchange
symbols = [
    'BTC/USDT:USDT',
    #'ETH/USDT:USDT',
    #'AAVE/USDT:USDT',
    #'BNB/USDT:USDT',
    #'CRV/USDT:USDT',
    #'SNX/USDT:USDT',
    #'LINK/USDT:USDT',
]
exchange_list = ['binance', 'huobi', 'okex']
data_df = pd.DataFrame(None, columns=header).set_index('Timestamp')

for exchange_name, exch in exchanges_dict.items():
    print(exch)
    if exchange_name not in exchange_list: continue
    if not exch.has["fetchOHLCV"]:
        print(f'-'*15+f'{exchange_name} does not support fetching OHLC data. Please use another exchange'+f'-'*15)
        quit()
    if (not hasattr(exch, 'timeframes')) or (t_frame not in exch.timeframes):
        print(f'-'*15+f'The requested timeframe ({t_frame}) is not available from {exchange_name}'+f'-'*15)
        print('Available timeframes are:')
        for key in exch.timeframes.keys(): print('  - ' + key)
        print('-'*80)
        quit()
    exch.load_markets()
    for coin in symbols:
        print(coin)
        since = exch.parse8601(start_date) # fetch from a certain starting datetime
        all_ohlcv = []
        while since < exch.milliseconds():
            coin_ohlcv = exch.fetchOHLCV(coin, t_frame, since)
            if not coin_ohlcv: break
            since = coin_ohlcv[-1][0] + 1 # query the last and add 1 to the number of milliseconds
            all_ohlcv += coin_ohlcv
        df = pd.DataFrame(all_ohlcv, columns=header).set_index('Timestamp')
        df['Symbol'] = coin
        df['Exchange'] = exch
        data_df = pd.concat([data_df, df])
        data_df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
        data_df['Date'] = pd.to_datetime(data_df.index, unit='s')
        data_df.to_csv(f"./data/{exch}_{start_date}_{t_frame}")
        data_df = pd.DataFrame()

Delta Exchange
Deribit
Binance
BTC/USDT:USDT
Huobi
BTC/USDT:USDT
OKX
BTC/USDT:USDT


In [46]:
### Delta Exchange BTC Index for Derivative
e_dict = {}
for e in ['Binance', 'Huobi', 'OKX']:
    e_dict[e] = pd.read_csv(f"./data/{e}_{start_date}_{t_frame}", index_col="Date")
    e_dict[e].index = pd.to_datetime(e_dict[e].index).strftime('%Y-%m-%d')

delta_index = pd.DataFrame()
delta_index['price'] = 0.33 * e_dict['Binance']['Close'] + 0.33 * e_dict['OKX']['Close'] + 0.33 * e_dict['Huobi']['Close']  

In [47]:
### Deribit BTC Index for Derivative
deribit_index = pd.read_csv(
    f"./data/2023-01_btc_usd_price_index.csv",
    usecols=['Date', 'Index']
)

deribit_index['Date'] = pd.to_datetime(deribit_index['Date'])
#pd.to_datetime(deribit_index.index)
deribit_index = deribit_index.set_index('Date')
daily_resampled_deribit_index = deribit_index.resample('D').agg({'Index': 'mean'})

In [48]:
# option['underlying']/option['quote']:option['quote']
# Compute the simple returns and multiply by 100 to avoid the scaling warning from arch_model.
# returns = 100 * df[df["Symbol"] == "BTC/USDT:USDT"]["Close"].pct_change().dropna()
delta_index_returns = 100 * delta_index['price'].pct_change().dropna()
deribit_index_returns = 100 * daily_resampled_deribit_index['Index'].pct_change().dropna()

delta_am = arch_model(delta_index_returns, vol="GARCH", p=1, o=0, q=1, dist="normal")
deribit_am = arch_model(deribit_index_returns, vol="GARCH", p=1, o=0, q=1, dist="normal")

delta_res = delta_am.fit(update_freq=5)
deribit_res = deribit_am.fit(update_freq=5)

delta_forecast = delta_res.forecast(horizon=int(delta_days_until_expiration)+1, reindex=False)
deribit_forecast = deribit_res.forecast(horizon=int(deribit_days_until_expiration)+1, reindex=False)

# forecast = res.forecast(horizon=1, reindex=False)
delta_variance_forecast = delta_forecast.variance.iloc[-1][0]
deribit_variance_forecast = deribit_forecast.variance.iloc[-1][0]

delta_volatility_forecast = np.sqrt(delta_variance_forecast)
deribit_volatility_forecast = np.sqrt(deribit_variance_forecast)

my_delta_annualized_volatility_forecast = delta_volatility_forecast * np.sqrt(252) / 100
my_deribit_annualized_volatility_forecast = deribit_volatility_forecast * np.sqrt(252) / 100

# Cette VOL est issu de la prediction du GARCH a comparer avec les IV de l'Option Wheel 
# et trade le mispricing
print(f"T_FRAME: {t_frame}")
pprint(my_delta_annualized_volatility_forecast)
pprint(my_deribit_annualized_volatility_forecast)

Iteration:      5,   Func. Count:     32,   Neg. LLF: 915.1281768757449
Optimization terminated successfully    (Exit mode 0)
            Current function value: 915.1273174401862
            Iterations: 8
            Function evaluations: 46
            Gradient evaluations: 8
Iteration:      5,   Func. Count:     38,   Neg. LLF: 2724.6139043341504
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2724.5675362167244
            Iterations: 9
            Function evaluations: 59
            Gradient evaluations: 9
T_FRAME: 1d
0.3461011599212809
0.2611076273215522


In [49]:
def price_range(iv, current_price, day_to_expiration):
    range_delta = float(current_price*iv*math.sqrt(day_to_expiration/365))
    return [round(current_price - range_delta,3), round(current_price + range_delta,3)]
### JUST AN EXAMPLE
price_range(my_delta_annualized_volatility_forecast, option_specification['strike'], days_until_expiration)

[23005.723, 23994.277]

In [50]:
exchanges_option_contract = {}
### DeltaExchange Format
"""
    Retrieve
    IV
    index price (Underlying price)
    days until expiration
    strike price
    interest rate
    dividend yield (0)
"""
delta_option_contract = tickers_dict['delta']
greeks = delta_option_contract["info"]["greeks"]
ask_iv = float(delta_option_contract["info"]["quotes"]['ask_iv'])
ask_size = float(delta_option_contract["info"]["quotes"]['ask_size'])
best_ask = float(delta_option_contract["info"]["quotes"]['best_ask'])
best_bid = float(delta_option_contract["info"]["quotes"]['best_bid'])
bid_iv = float(delta_option_contract["info"]["quotes"]['bid_iv'])
bid_size = float(delta_option_contract["info"]["quotes"]['bid_size'])
mark_iv = float(delta_option_contract["info"]["quotes"]['mark_iv'])
spot_price = float(delta_option_contract["info"]["spot_price"])

exchanges_option_contract.update(
    {
        'delta': {
            'spot_price': spot_price,
            'best_ask': best_ask,
            'ask_iv': ask_iv,
            'best_bid': best_bid,
            'bid_iv': bid_iv,
            'expiration': delta_days_until_expiration,
            'payoff': option_specification['payoff'],
            'my_iv': my_delta_annualized_volatility_forecast,
        }
    }
)

In [51]:
### Deribit Format
deribit_option_contract = tickers_dict['deribit']
greeks = deribit_option_contract["info"]["greeks"] # Deribit format
ask_iv = float(deribit_option_contract['info']['ask_iv']) if float(deribit_option_contract['info']['ask_iv']) else float(deribit_option_contract['info']['mark_iv']) 
strike_price = option_specification['strike']
spot_price = float(deribit_option_contract["info"]["underlying_price"])
best_ask_price = float(deribit_option_contract["info"]['best_ask_price']) if float(deribit_option_contract['info']['best_ask_price']) else float(deribit_option_contract['info']['mark_price']) 
best_bid_price = float(deribit_option_contract["info"]['best_bid_price']) if float(deribit_option_contract['info']['best_bid_price']) else float(deribit_option_contract['info']['mark_price']) 
ask_iv = float(deribit_option_contract['info']['bid_iv']) if float(deribit_option_contract['info']['bid_iv']) else float(deribit_option_contract['info']['mark_iv']) 

exchanges_option_contract.update(
    {
        'deribit': {
            'spot_price': spot_price,
            'best_ask': best_ask_price*spot_price,
            'ask_iv': ask_iv/100.0,
            'best_bid': best_bid_price*spot_price,
            'bid_iv': bid_iv/100.0, 
            'expiration': deribit_days_until_expiration,   
            'payoff': option_specification['payoff'],  
            'my_iv': my_deribit_annualized_volatility_forecast,   
        }
    }
)

In [52]:
pprint(f"Result for {contract_specification}")

for exch, option  in exchanges_option_contract.items():
   print(f"{'='*20} {exch.upper()} SIDE IS {contract_specification['side']} {'='*20}")
   if contract_specification["side"] == "ask":
      info = {
         "S0": float(option['spot_price']),
         "K": float(option_specification['strike']),
         "T": float(option['expiration']/365),
         "r": float(1/100),
         "sigma": float(option['ask_iv']),
         "payoff": option_specification["payoff"],
      }
      # pprint(f"{info['sigma']:.03f}")
      # BS CLASSIC WITH IV
      theorical_option_price = BS_pricer.BlackScholes(**info)
      # BS CLASSIC WITH GARCH IV
      my_option_price = BS_pricer.BlackScholes(
         S0=float(option['spot_price']),
         K=float(option_specification['strike']),
         T=float(option['expiration']/365),
         r=float(1/100),
         sigma=float(option['my_iv']),
         payoff=option_specification["payoff"]     
      )
      # pprint(f"{float(my_annualized_volatility_forecast):.03f}")
      # BIN PRICING WITH THEIR IV   
      bin_price = binomial_price(**info)

      # PRINT
      print(f"Theorical price, Their IV: {theorical_option_price:.03f}, {float(option['ask_iv'])*100:.03f}%")
      print(f"Price Range: {price_range(float(option['ask_iv']), float(option['spot_price']), float(option['expiration']/365),)}")
      print(f"GARCH price, My IV: {my_option_price:.03f}, {float(option['my_iv'])*100:.03f}%")
      print(f"Price Range: {price_range(float(option['my_iv']), float(option['spot_price']), float(option['expiration']/365),)}")
      print(f"binomial price: {bin_price:.03f}") 
      print(f"current market price {option['best_ask']:.03f}")
   elif contract_specification["side"] == "bid":
      info = {
         "S0": float(option['spot_price']),
         "K": float(option_specification['strike']),
         "T": float(option['expiration']/365),
         "r": float(1/100),
         "sigma": float(option['bid_iv']),
         "payoff": option_specification["payoff"],
      }
      # pprint(f"{info['sigma']:.03f}")
      # BS CLASSIC WITH IV
      theorical_option_price = BS_pricer.BlackScholes(**info)
      # BS CLASSIC WITH GARCH IV
      my_option_price = BS_pricer.BlackScholes(
         S0=float(option['spot_price']),
         K=float(option_specification['strike']),
         T=float(option['expiration']/365),
         r=float(1/100),
         sigma=float(option['my_iv']),
         payoff=option_specification["payoff"]     
      )
      # pprint(f"{float(my_annualized_volatility_forecast):.03f}")
      # BIN PRICING WITH THEIR IV   
      bin_price = binomial_price(**info)

      # PRINT
      print(f"Theorical price, Their IV: {theorical_option_price:.03f}, {float(option['bid_iv'])*100:.03f}%")
      print(f"Price Range: {price_range(float(option['bid_iv']), float(option['spot_price']), float(option['expiration']/365),)}")
      print(f"GARCH price, My IV: {my_option_price:.03f}, {float(option['my_iv'])*100:.03f}%")
      print(f"Price Range: {price_range(float(option['my_iv']), float(option['spot_price']), float(option['expiration']/365),)}")
      print(f"binomial price: {bin_price:.03f}") 
      print(f"current market price {option['best_bid']:.03f}")   

print(f"="*(42+len(max(list(exchanges_dict.keys())))))

("Result for {'expiration': '230128', 'payoff': 'C', 'underlying': 'BTC', "
 "'strike': 23500, 'quote': 'USDT', 'side': 'bid'}")
==================== DELTA SIDE IS bid ====================
Theorical price, Their IV: 187.085, 49.026%
Price Range: [23119.413, 23204.945]
GARCH price, My IV: 101.320, 34.610%
Price Range: [23131.988, 23192.37]
binomial price: 187.085
current market price 185.000
==================== DERIBIT SIDE IS bid ====================
Theorical price, Their IV: 0.000, 0.490%
Price Range: [23183.493, 23184.301]
GARCH price, My IV: 53.176, 26.111%
Price Range: [23162.392, 23205.402]
binomial price: 0.000
current market price 208.655


In [53]:
### DATA RETRIEVE BACKUP

for exch in exchange_list:
    print(exch)
    try:
        exchange = getattr(ccxt, exch)()
    except AttributeError:
        print('-'*36,' ERROR ','-'*35)
        print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
        print('-'*80)
        quit()
    if exchange.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print('{} does not support fetching OHLC data. Please use another exchange'.format(exch))
        print('-'*80)
        quit()
    if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
        print('Available timeframes are:')
        for key in exchange.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exchange.load_markets()
    for coin in symbols:
        print(coin)
        since = exchange.parse8601(start_date) # fetch from a certain starting datetime
        all_ohlcv = []
        while since < exchange.milliseconds():
            coin_ohlcv = exchange.fetchOHLCV(coin, t_frame, since)
            if not coin_ohlcv: break
            since = coin_ohlcv[-1][0] + 1 # query the last and add 1 to the number of milliseconds
            all_ohlcv += coin_ohlcv
        df = pd.DataFrame(all_ohlcv, columns=header).set_index('Timestamp')
        df['Symbol'] = coin
        df['Exchange'] = exch
        data_df = pd.concat([data_df, df])

data_df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
data_df['Date'] = pd.to_datetime(data_df.index, unit='s')
data_df.to_csv(f"./data/{start_date}_{t_frame}")

binance
BTC/USDT:USDT
huobi


KeyboardInterrupt: 